In [118]:
from google.cloud import documentai
from google.oauth2 import service_account
from google.auth import load_credentials_from_file
from google.cloud import aiplatform
import vertexai
from vertexai.language_models import TextGenerationModel
from vertexai.generative_models import GenerativeModel, SafetySetting, Part
import json
import os
import openai
import timeit
from scipy.spatial.distance import cosine
import time
import pandas as pd
import numpy as np

In [137]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [20]:
with open('./Test Data/test_data.json', "r") as json_file:
    test_data = json.load(json_file)

In [5]:
generation_config = {
    "max_output_tokens": 1024,
    "temperature": 0.2,
    "top_p": 1,
}

safety_settings = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
]


In [72]:
credentials, project_id = load_credentials_from_file("./GSuite Text Extraction Creds/vertex_ai_key.json")
vertexai.init(credentials=credentials,project="90458358443", location="us-central1")


In [97]:
model = GenerativeModel(
    "projects/90458358443/locations/us-central1/endpoints/352768410167279616",
    system_instruction=["You are a helpful tutor for the class - Applied Large Language Models and Natural Language Processing"]
)

In [98]:
model.generate_content(
    "What is a RAG system"
)

candidates {
  content {
    role: "model"
    parts {
      text: "RAG system stands for Retrieval Augmented Generation, which is a system that combines"
    }
  }
  finish_reason: STOP
  avg_logprobs: -1.0132228306361608
}
usage_metadata {
  prompt_token_count: 22
  candidates_token_count: 14
  total_token_count: 36
}

<h1>Creating Embedding Similarity Mechanism</h1>

In [44]:
def get_embedding(text, model="text-embedding-3-large"):
    response = openai.Embedding.create(input=text, model=model)
    return response['data'][0]['embedding']

In [45]:
def cosine_similarity(vec1, vec2):
    return 1 - cosine(vec1, vec2)


In [135]:
all_messages = [
{"role": "system", "content": "You are a helpful assistant."},
{"role": "user", "content": f"Write a story about a magic backpack."}
]
response = openai.ChatCompletion.create(
model="ft:gpt-4o-mini-2024-07-18:f-prime-capital::AbZYSjIT",
max_tokens = 1500,
messages=all_messages
)

In [136]:
response['choices'][0]['message']['content']

'Once upon a time, in a small town, there lived a young girl named Ellie who loved to explore. One sunny morning, her grandmother gifted her an old backpack adorned with mysterious symbols. Little did Ellie know, this magic backpack had the power to carry everything she could ever need for an adventure!'

In [124]:
# Evaluate semantic similarity
similarities = []
start = timeit.default_timer()
# last_pause = timeit.default_timer()
for i,example in enumerate(test_data):
    try:
        input_text = example['input']
        expected_output = example['output']
        # Generate model response
        all_messages = [
        {"role": "system", "content": "You are a helpful tutor who answers questions about a class called Introduction to Deep Learning and LLM based Generative AI Systems"},
        {"role": "user", "content": f"Please answer this question: {input_text}"}
        ]
        response = openai.ChatCompletion.create(
        model="ft:gpt-4o-mini-2024-07-18:f-prime-capital::AbZYSjIT",
        max_tokens = 1500,
        messages=all_messages
        )
        # print(model_response)
        model_response_text = response['choices'][0]['message']['content']
        # Generate embeddings for expected and actual responses
        expected_embedding = get_embedding(expected_output)
        model_response_embedding = get_embedding(model_response_text)

        # Calculate similarity
        similarity = cosine_similarity(expected_embedding, model_response_embedding)
        similarities.append({'question':input_text ,'expected_output':expected_output,'model_output':model_response_text,'similarities':similarity})
    except Exception as e:
        print(i,e)
        # break
    if (i%5==0)&(i!=0):
        end = timeit.default_timer()
        print(f"{i} - Time Spent: {end-start}, Number of Errors: {i + 1 - len(similarities)}")
        start = timeit.default_timer()
        # print(f"Sleeping for {60-(timeit.default_timer()-last_pause)} seconds")
        # if (60-(timeit.default_timer()-last_pause))>0:
        #     time.sleep(60-(timeit.default_timer()-last_pause))
        # last_pause = timeit.default_timer()



5 - Time Spent: 9.12100158398971, Number of Errors: 0
10 - Time Spent: 6.962685958947986, Number of Errors: 0
15 - Time Spent: 6.2417064579203725, Number of Errors: 0
20 - Time Spent: 6.457824958954006, Number of Errors: 0
25 - Time Spent: 6.607852041721344, Number of Errors: 0
30 - Time Spent: 6.134849541820586, Number of Errors: 0
35 - Time Spent: 7.180876875296235, Number of Errors: 0
40 - Time Spent: 7.026023915968835, Number of Errors: 0
45 - Time Spent: 6.691054415889084, Number of Errors: 0
50 - Time Spent: 6.708624749910086, Number of Errors: 0
55 - Time Spent: 6.998260000254959, Number of Errors: 0
60 - Time Spent: 8.474515459034592, Number of Errors: 0
65 - Time Spent: 5.982142584398389, Number of Errors: 0
70 - Time Spent: 7.698597500100732, Number of Errors: 0
75 - Time Spent: 6.875113124959171, Number of Errors: 0
80 - Time Spent: 6.796475583687425, Number of Errors: 0
85 - Time Spent: 6.660923125222325, Number of Errors: 0
90 - Time Spent: 6.649852499831468, Number of Err

In [125]:
open_ai_fine_tuned_tested_data = pd.DataFrame(similarities)

In [126]:
# Evaluate semantic similarity
similarities = []
start = timeit.default_timer()
# last_pause = timeit.default_timer()
for i,example in enumerate(test_data):
    try:
        input_text = example['input']
        expected_output = example['output']
        # Generate model response
        all_messages = [
        {"role": "system", "content": "You are a helpful tutor who answers questions about a class called Introduction to Deep Learning and LLM based Generative AI Systems"},
        {"role": "user", "content": f"Please answer this question: {input_text}"}
        ]
        response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        max_tokens = 1500,
        messages=all_messages
        )
        # print(model_response)
        model_response_text = response['choices'][0]['message']['content']
        # Generate embeddings for expected and actual responses
        expected_embedding = get_embedding(expected_output)
        model_response_embedding = get_embedding(model_response_text)

        # Calculate similarity
        similarity = cosine_similarity(expected_embedding, model_response_embedding)
        similarities.append({'question':input_text ,'expected_output':expected_output,'model_output':model_response_text,'similarities':similarity})
    except Exception as e:
        print(i,e)
        # break
    if (i%5==0)&(i!=0):
        end = timeit.default_timer()
        print(f"{i} - Time Spent: {end-start}, Number of Errors: {i + 1 - len(similarities)}")
        start = timeit.default_timer()
        # print(f"Sleeping for {60-(timeit.default_timer()-last_pause)} seconds")
        # if (60-(timeit.default_timer()-last_pause))>0:
        #     time.sleep(60-(timeit.default_timer()-last_pause))
        # last_pause = timeit.default_timer()



5 - Time Spent: 27.011017416138202, Number of Errors: 0
10 - Time Spent: 20.23899329220876, Number of Errors: 0
15 - Time Spent: 19.805863584391773, Number of Errors: 0
20 - Time Spent: 15.374278917443007, Number of Errors: 0
25 - Time Spent: 23.608335040975362, Number of Errors: 0
30 - Time Spent: 9.505918709095567, Number of Errors: 0
35 - Time Spent: 16.075823791790754, Number of Errors: 0
40 - Time Spent: 14.458874749951065, Number of Errors: 0
45 - Time Spent: 17.606029166840017, Number of Errors: 0
50 - Time Spent: 15.585122166667134, Number of Errors: 0
55 - Time Spent: 17.16982129216194, Number of Errors: 0
60 - Time Spent: 17.3189439997077, Number of Errors: 0
65 - Time Spent: 21.20832766685635, Number of Errors: 0
70 - Time Spent: 26.695609542075545, Number of Errors: 0
75 - Time Spent: 23.17311262479052, Number of Errors: 0
80 - Time Spent: 12.37620708392933, Number of Errors: 0
85 - Time Spent: 23.58480229228735, Number of Errors: 0
90 - Time Spent: 25.35080008301884, Numbe

In [127]:
open_ai_base_line_tested_data = pd.DataFrame(similarities)

In [131]:
open_ai_base_line_tested_data['similarities'].mean()

np.float64(0.7260634112120203)

In [132]:
open_ai_base_line_tested_data['similarities'].std()

np.float64(0.1248041682966205)